##Data

In [ ]:
!pip install mediapipe

In [ ]:
import cv2 # type: ignore
import numpy as np # type: ignore
import os
from matplotlib import pyplot as plt # type: ignore
import time
import mediapipe as mp # type: ignore

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://github.com/Al-ameen007/SSL/raw/main/MP_Data.zip

In [ ]:
!unzip /content/MP_Data.zip

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = '/content/MP_Data'

# Actions that we try to detect
actions = np.array(['Ahlan', 'Shokran', 'Asef'])

# Thirty videos worth of data
no_sequences = 120

# Videos are going to be 30 frames in length
sequence_length = 30

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # type: ignore
label_map = {label:num for num, label in enumerate(actions)}
label_map
{'ahlan': 0, 'asef': 1, 'shokran': 2}
sequences, labels = [], []

In [ ]:
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(288, 30, 1662) (72, 30, 1662) (288, 3) (72, 3)


##Model

In [ ]:
from tensorflow.keras.models import Sequential  # type: ignore
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, BatchNormalization # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.callbacks import LearningRateScheduler # type: ignore
from tensorflow.keras.callbacks import TensorBoard # type: ignore
import numpy as np

# Define a custom learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 100:
        return lr  # Keep the initial learning rate for the first 100 epochs
    else:
        return lr * np.exp(-0.1)  # Exponentially decrease the learning rate after 100 epochs

In [ ]:
# Define the model
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True, activation='relu'), input_shape=(30, 1662)))
model.add(Dropout(0.2))  # Add dropout for regularization
model.add(BatchNormalization())  # Add batch normalization to stabilize training
model.add(Bidirectional(LSTM(128, return_sequences=True, activation='relu')))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(64, return_sequences=False, activation='relu')))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
# Compile the model with a custom learning rate scheduler
adam_optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define a tensorboard callback for visualization
tb_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

# Train the model with the defined callbacks and custom learning rate scheduler
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback, LearningRateScheduler(lr_scheduler)],  verbose=0)

In [ ]:
acc = 0
for i in range(len(X_test)):
  res = model.predict(X_test, verbose=0)
  if np.argmax(res[i]) == np.argmax(y_test[i]):
    acc+=1
  print(actions[np.argmax(res[i])], actions[np.argmax(y_train[i])])

print(acc/len(X_train))

Shokran Shokran
Ahlan Ahlan
Asef Ahlan
Ahlan Asef
Asef Ahlan
Shokran Shokran
Asef Shokran
Shokran Asef
Shokran Shokran
Asef Asef
Shokran Ahlan
Asef Asef
Ahlan Ahlan
Ahlan Shokran
Ahlan Asef
Asef Shokran
Asef Shokran
Ahlan Asef
Asef Ahlan
Shokran Ahlan
Asef Asef
Shokran Ahlan
Ahlan Shokran
Asef Asef
Ahlan Asef
Asef Ahlan
Asef Shokran
Shokran Shokran
Ahlan Asef
Shokran Ahlan
Shokran Ahlan
Asef Ahlan
Asef Ahlan
Shokran Ahlan
Asef Shokran
Shokran Shokran
Asef Asef
Ahlan Asef
Asef Asef
Asef Asef
Ahlan Ahlan
Asef Shokran
Asef Asef
Asef Ahlan
Ahlan Shokran
Asef Shokran
Shokran Asef
Shokran Ahlan
Shokran Ahlan
Ahlan Shokran
Asef Asef
Shokran Asef
Ahlan Asef
Shokran Shokran
Shokran Ahlan
Shokran Shokran
Shokran Asef
Shokran Ahlan
Ahlan Shokran
Shokran Ahlan
Ahlan Shokran
Asef Ahlan
Ahlan Shokran
Ahlan Asef
Ahlan Shokran
Shokran Ahlan
Ahlan Ahlan
Asef Shokran
Asef Ahlan
Asef Shokran
Ahlan Shokran
Ahlan Shokran
0.25


In [ ]:
model.save('action.h5')